In [1]:
from keras.datasets import mnist
from sklearn.utils import shuffle
import math
import random
import numpy as np


import torch

if torch.cuda.is_available():  
  dev = "cuda:0" 
  print("gpu up")
else:  
  dev = "cpu"  
device = torch.device(dev)


(train_X, train_y), (test_X, test_y) = mnist.load_data()


Using TensorFlow backend.


gpu up


In [6]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()


def centroid(arr):
    moments = {'M00': arr.sum(axis=1).sum(),
               'M01': (np.arange(start=1, stop=arr.shape[1] + 1, step=1) * arr.sum(axis=0).reshape(1, -1)).sum(),
               'M10': (np.arange(start=1, stop=arr.shape[0] + 1, step=1) * arr.sum(axis=1).reshape(1, -1)).sum(),
               'M11': 0}

    x = moments['M10'] / moments['M00'] if moments['M00'] != 0 else 0
    y = moments['M01'] / moments['M00'] if moments['M00'] != 0 else 0
    return [x, y]

threshold = 150  # for Binarize


def Binarize(X):
    X[X < threshold] = 0
    X[X >= threshold] = 1
    return X


train_X = Binarize(train_X)
test_X = Binarize(test_X)


def normalize(X):
    mx = np.max(X)
    return X/mx


def imageWin(trainx, r, c):
    temp = []
    for i in range(0, int(trainx.shape[0]), r):
        for j in range(0, int(trainx.shape[1]), c):
            temp.append(centroid(trainx[i:i+r, j:j+c]))
    return np.array(temp)


def slicingArray(trainX, r, c):
    featureVector = []
    for trainx in trainX:
        featureVector.append(imageWin(trainx, r, c))
    return np.array(featureVector)


In [7]:
# Some Fundamental Functions
def listXlist(l1: list, l2: list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l1))]
    # print(len(l1),len(l1[0])," ",len(l2),len(l2[0]))
    for i in range(len(l1)):
        for j in range(len(l2[0])):
            for k in range(len(l2)):
                print(res[i][j])
                res[i][j] += l1[i][k] * l2[k][j]
    return res


def listMINUSlist(l1: list, l2: list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l2))]
    for i in range(len(l2)):
        for j in range(len(l1[0])):
            res[i][j] = l1[i][j] - l2[i][j]
    return res


def listSquare(l: list):
    res = []
    for i in range(len(l)):
        res.append([l[i]**2])
    return res


def mean(k: list):
    return sum([i[0] for i in k]) / len(k)


def get_label(vector: list):
    mx = 0
    idx = 0
    for i in range(len(vector)):
        if(vector[i] > mx):
            mx = vector[i]
            idx = i
    return idx


def normalizeOut(vector: list):
    mx = vector[get_label(vector)]
    vector = [vector[i]/mx for i in range(len(vector))]
    return vector


def accuracy(target: list, predictions: list):
    trueValues = [get_label(target[i]) for i in range(len(target))]
    preds = [get_label(predictions[i]) for i in range(len(predictions))]
    cnt = 0
    for i in range(len(trueValues)):
        if trueValues[i] == preds[i]:
            cnt += 1

    return cnt / float(len(trueValues)) * 100.0


In [8]:
# Shuffling the dataset

# train_X, train_y = shuffle(train_X[:1500], train_y[:1500])
# test_X, test_y = shuffle(test_X[:100], test_y[:100])

train_X, train_y = train_X[:500], train_y[:500]
test_X, test_y = test_X[:100], test_y[:100]

arr = slicingArray(train_X, 4, 7)
trainFeatureVec = normalize(arr)
trainFeatureVec = trainFeatureVec.reshape([*arr.shape[:-2], -1])
trainFeatureVec = np.expand_dims(trainFeatureVec, axis=-1)


# print(trainFeatureVec.shape)

trainFeatureVec = trainFeatureVec.tolist()


arr = slicingArray(test_X, 4, 7)
testFeatureVec = normalize(arr)
testFeatureVec = testFeatureVec.reshape(*arr.shape[:-2], -1)
testFeatureVec = np.expand_dims(testFeatureVec, axis=-1)


train_y = (np.eye(10)[np.array(train_y)])
test_y = (np.eye(10)[np.array(test_y)])
train_y = np.expand_dims(train_y, axis=-1).tolist()
test_y = np.expand_dims(test_y, axis=-1).tolist()


In [9]:
# Some Fundamental Functions
def listXlist(l1: list, l2: list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l1))]
    # print(len(l1),len(l1[0])," ",len(l2),len(l2[0]))
    for i in range(len(l1)):
        for j in range(len(l2[0])):
            for k in range(len(l2)):
                res[i][j] += l1[i][k] * l2[k][j]
    return res


def listMINUSlist(l1: list, l2: list):
    res = [[0 for i in range(len(l2[0]))] for j in range(len(l2))]
    for i in range(len(l2)):
        for j in range(len(l1[0])):
            res[i][j] = l1[i][j] - l2[i][j]
    return res


def listSquare(l: list):
    res = []
    for i in range(len(l)):
        res.append([l[i]**2])
    return res


def mean(k: list):
    return sum([i[0] for i in k]) / len(k)


def get_label(vector: list):
    mx = 0
    idx = 0
    for i in range(len(vector)):
        if(vector[i] > mx):
            mx = vector[i]
            idx = i
    return idx


def normalizeOut(vector: list):
    mx = vector[get_label(vector)]
    vector = [vector[i]/mx for i in range(len(vector))]
    return vector


def accuracy(target: list, predictions: list):
    trueValues = [get_label(target[i]) for i in range(len(target))]
    preds = [get_label(predictions[i]) for i in range(len(predictions))]
    cnt = 0
    for i in range(len(trueValues)):
        if trueValues[i] == preds[i]:
            cnt += 1

    return cnt / float(len(trueValues)) * 100.0



In [73]:
# Fully connected NN
class NN:
    def __init__(self, m: int, n: list, activation: str, e: int, lr: float, outShape: int):
        self.neurons = n                                                    # Neurons List to specify number of neurons in each layer
        self.neuronsListBefore = []                                         # Net values
        self.neuronsListAfter = []                                          # Neurons output values
        self.layers = m                                                     # Number of layers
        self.weights = dict([(x, [0]) for x in range(self.layers + 1)])     # Dictionary to save weights of each layer
        self.bias = dict([(x, [0]) for x in range(self.layers + 1)])        # Dictionary to save bias of each layer
        self.gradsWeights = []                                              # Dictionary to save weights gradients of each layer
        self.gradsBias = []                                                 # Dictionary to save bias gradients of each layer
        self.activationFn = activation                                      # String to specify layers' activation function
        self.learning_rate = lr                                         
        self.epochs = e
        self.outputShape = outShape                                         # Number of neurons in output layer

    # MSE
    def __costFunc(self, y, preds):
        return 0.5 * listSquare(listMINUSlist(y, preds)) /len(y)

    def __activation(self, x: float, act: str):
        # if act == 'sigmoid':
        #     if x > 20:
        #         return 0.00000001
        #     elif x < -20:
        #         return 0.99999999
       
        if act == "sigmoid":
            return 1 / (1 + np.exp(-x))
        if act == "tanh":
            return np.tanh(x)

        if act == "linear":
            return x
        if act=="relu":
            return np.maximum(x, 0),
        if act=="leaky_relu":
            return np.maximum(0.1 * x, x)
        
        # return res[act]

    def __derivatives(self, out: float, act: str):
        if act == "sigmoid":
            return out * (1-out)
        if act=="tanh":
            # print(out)
            return 1 - (out ** 2)

        if act=="linear":
            return np.ones(out.shape)
        if act=="relu":
            re = np.ones(out.shape)
            re[out<0] = np.zeros((out<0).shape)
            return re
        if act=="leaky_relu":
            re = np.ones(out.shape)
            re[out<0] = out[out<0] * 0.1
            return re
        

    def __initializeWeightsBias(self, input: list):
        self.weights[0] = np.array([[random.random() for i in range(len(input))]
                           for j in range(self.neurons[0])],dtype=np.double)
        self.bias[0] = np.array([[random.random()]
                        for i in range(self.neurons[0])],dtype=np.double)

        for i in range(1, self.layers):
            self.weights[i] = np.array([[random.random() for i in range(
                self.neurons[i - 1])] for j in range(self.neurons[i])],dtype=np.double)
            self.bias[i] = np.array([[random.random()]
                            for i in range(self.neurons[i])],dtype=np.double)

        self.weights[self.layers] = np.array([[random.random() for i in range(
            self.neurons[self.layers - 1])] for j in range(self.outputShape)],dtype=np.double)
        self.bias[self.layers] = np.array([[random.random()]
                                  for i in range(self.outputShape)],dtype=np.double)

    def __zero_grads(self, input: list):
        self.gradsWeights = dict([(x, [0]) for x in range(self.layers + 1)])
        self.gradsBias = dict([(x, [0]) for x in range(self.layers + 1)])
        self.gradsWeights[0] = np.array([[0 for i in range(len(input))]
                                for j in range(self.neurons[0])],dtype=np.double)
        self.gradsBias[0] = np.array([[0] for i in range(self.neurons[0])],dtype=np.double)

        for i in range(1, self.layers):
            self.gradsWeights[i] = np.array([
                [0 for _ in range(self.neurons[i - 1])] for j in range(self.neurons[i])],dtype=np.double)
            self.gradsBias[i] = np.array([[0] for i in range(self.neurons[i])],dtype=np.double)

        self.gradsWeights[self.layers] = np.array([[0 for i in range(
            self.neurons[self.layers - 1])] for j in range(self.outputShape)],dtype=np.double)
        self.gradsBias[self.layers] = np.array([[0] for i in range(self.outputShape)],dtype=np.double)

    def __feedForward(self, input: list, layerIdx: int) -> list:
        out = []
        x = []
        # print(input,self.weights[layerIdx])
        # print(layerIdx)
        # print(input.shape)
        # print(input)
        x = (self.weights[layerIdx]@ input) + self.bias[layerIdx]
        # x = [[x[i][0] + self.bias[layerIdx][i]] for i in range(len(x))]
        # x = 
        self.neuronsListBefore.append(x)
        # for i in range(len(x)):
        #     out.append([self.__activation(x[i][0], self.activationFn)])
        out = self.__activation(x, self.activationFn)
        # print(out, "<<<<<<<<<<<<<<<<<<<<<<")
        self.neuronsListAfter.append(out)
        return out

    def __propagate(self, x: list):
        self.neuronsListAfter.append(x)
        out = self.__feedForward(x, 0)
        for i in range(1, self.layers + 1):
            out = self.__feedForward(out, i)
        return out

    def __backPropagation(self, target: list, outputLayer: list):
        # print(len(outputLayer),outputLayer)
        # dw = -1 * mean(listMINUSlist(target, outputLayer))
        deltas = dict([(x, [0]) for x in range(self.layers + 1)])
        # deltas[self.layers] = np.array(
        #     [k[0] for k in listMINUSlist(target, outputLayer)])/-10
        deltas[self.layers] = ( -1* (target-outputLayer)/10 )* (self.__derivatives(
                self.neuronsListBefore[self.layers], self.activationFn))
        self.gradsBias[self.layers]= deltas[self.layers]

        # (neurons , 1) @ (1,neuronsBefore)   -> (neurons, neuronsBefore)
        # self.gradsWeights[self.layers][i][j] = deltas[self.layers][i] * \
        #             self.neuronsListAfter[self.layers][j][0]
        self.gradsWeights[self.layers] = deltas[self.layers] @ self.neuronsListAfter[self.layers].T
       
               

        # [sum([self.weights[j][i] * delta[layer + 1][j] for j in range(len(delta[layer]))]) for i in range(len(weights))]

        for layer in reversed(range(0, self.layers)):

            # (neuron, 1) (neuron, neuronsBefore ) -> (neurons,Before)
            deltas[layer] = (self.weights[layer + 1].T @ deltas[layer+1] )* self.__derivatives(
                    self.neuronsListBefore[layer], self.activationFn)
            self.gradsBias[layer] = deltas[layer]
            self.gradsWeights[layer] = deltas[layer] @ self.neuronsListAfter[layer].T
            
            # deltas[layer] = [sum([self.weights[layer + 1][j][i] * deltas[layer + 1][j]
            # #                      for j in range(len(deltas[layer]))]) for i in range(len(self.weights[layer + 1][0]))]
            # for i in range(len(self.weights[layer])):
            #     # deltas[layer][i] *= self.__derivatives(
            #     #     self.neuronsListBefore[layer][i][0], self.activationFn)
            #     # # print(deltas)
            #     for j in range(len(self.weights[layer][0])):
            #         # print(layer, i,j)
            #         self.gradsWeights[layer][i][j] = deltas[layer][i] * \
            #             self.neuronsListAfter[layer][j][0]

    def __updateWeightsBias(self):

        for layer in range(self.layers + 1):
            self.bias[layer] -= self.learning_rate * self.gradsBias[layer]
            self.weights[layer] -= self.learning_rate * self.gradsWeights[layer] 
            self.weights[layer] -= self.learning_rate * self.weights[layer] *0.1
            # for i in range(len(self.weights[layer])):
            #     self.bias[layer][i] -= (self.learning_rate *
            #                             self.gradsBias[layer][i])
            #     for j in range(len(self.weights[layer][0])):
            #         self.weights[layer][i][j] -= (self.gradsWeights[layer]
            #                                       [i][j] * self.learning_rate)

    def fit(self, trainX: list, trainY: list):
        self.__initializeWeightsBias(trainX[0])
        self.__zero_grads(trainX[0])
        for epoch in range(self.epochs):
            for i in range(len(trainX)):
                self.neuronsListBefore = []
                self.neuronsListAfter = []
                outputLayer = self.__propagate(trainX[i])
                self.__backPropagation(trainY[i], outputLayer)
                self.__updateWeightsBias()
                self.__zero_grads(trainX[i])

    def predict(self, testX: list):
        preds = []
        for i in range(len(testX)):
            preds.append(self.__propagate(testX[i]))
        return preds


In [1]:
%pip install -U ipykernel

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\traitlets-5.0.5.dist-info\\COPYING.md'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.0.5
    Uninstalling traitlets-5.0.5:


In [77]:
nn = NN(2, [56,16], 'sigmoid', 5, 0.003, 10)
nn.fit(np.array(trainFeatureVec,dtype=np.float64), train_y)

testpreds = nn.predict(np.array(trainFeatureVec,dtype=np.float64))

<ipython-input-73-f6b1cdf7e204>:125: RuntimeWarning: overflow encountered in multiply
  deltas[self.layers] = ( -1* (target-outputLayer)/10 )* (self.__derivatives(
<ipython-input-73-f6b1cdf7e204>:144: RuntimeWarning: invalid value encountered in matmul
  self.gradsWeights[layer] = deltas[layer] @ self.neuronsListAfter[layer].T
<ipython-input-73-f6b1cdf7e204>:162: RuntimeWarning: invalid value encountered in subtract
  self.weights[layer] -= self.learning_rate * self.weights[layer]


In [64]:
testpreds = nn.predict(np.array(testFeatureVec))

In [65]:
sum([np.array(testpreds[i]).reshape(-1).argmax() ==np.array(train_y[i]).reshape(-1).argmax() for i in range(500)])/500

IndexError: list index out of range